In [ ]:
config = {
    "pretrained_model": "bert-base-cased",
    "tokenizer": "bert-base-cased",
    "max_seq_length": 256,
    "batch_size": 16,
    "lr": 2e-5,
    "epochs": 10,
    "device": "cuda",
    "gpu_ids": "2",
    "seed": 2020,
    "fp16": False,
    "loss_scale": 0,
    "gradient_accumulation_steps":1,
    "warmup_proportion": 0.1,
    "gradient_accumulation_steps": 1,
    "num_labels": 4,
    "is_multilabel": False,
    "valid_metric": "macro_f1",
    "model_save_dir": "../checkpoints/bert_cased_256_biocaster_4cate_20200903/",
    "patience": 4,
}

In [2]:
import sys
sys.path.append("../")
from beta_nlp.utils.data_util import biocaser2text,biocaster2df
data_file = "/home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml"
data_df = biocaster2df(data_file)
data_df["source"] = "Biocaster"
data_df.head(3)

parse biocaser data from /home/zm324/workspace/doc_cls/datasets/biocaster/BioCaster.3.xml, docs number:1003, lablels number:1003


,docs,labels,source
0,\nBird Flu Outbreak Drill Spooks Manitoba Town...,0,Biocaster
1,\nTyphoid outbreak in Agusan del Sur town unde...,3,Biocaster
2,\n Typhoid Outbreak In Central Nepal November...,3,Biocaster


In [3]:
from sklearn.utils import shuffle
data_df = shuffle(data_df).reset_index()
data_df["flag"]=None
train_index = int(len(data_df.index)*0.8)
valid_index = train_index + round(len(data_df.index)*0.1)
data_df.iloc[:train_index]["flag"]="train"
data_df.iloc[train_index:valid_index]["flag"]="valid"
data_df.iloc[valid_index:]["flag"]="test"

<ipython-input-3-79406d09754d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[:train_index]["flag"]="train"
<ipython-input-3-79406d09754d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_df.iloc[train_index:valid_index]["flag"]="valid"
<ipython-input-3-79406d09754d>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [4]:
train_set = data_df[data_df["flag"]=="train"]
dev_set = data_df[data_df["flag"]=="valid"]
test_set = data_df[data_df["flag"]=="test"]

In [5]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [6]:
from beta_nlp.models.bert_cls import BertModel
cls = BertModel(config)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (28996, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [7]:
len(train_set.index),len(dev_set.index),len(test_set.index)

(802, 100, 101)

In [8]:
cls.train(train_set,dev_set)

Evaluating:  29%|██▊       | 2/7 [00:00<00:00, 10.78it/s]

Execute [train_an_epoch] method costing 14217.95 ms
[Epoch 0] loss: 58.540070950984955



Evaluating: 100%|██████████| 7/7 [00:00<00:00, 11.91it/s]
/home/zm324/anaconda3/envs/doc_cls/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Execute [eval] method costing 593.47 ms
------------------------------------------------------------
[Epoch 0]performance on validation set
+----+-----------------+----------+
|    | metrics         |   values |
|----+-----------------+----------|
|  0 | accuracy        | 0.62     |
|  1 | macro_precision | 0.155    |
|  2 | macro_recall    | 0.25     |
|  3 | macro_f1        | 0.191358 |
|  4 | micro_precision | 0.62     |
|  5 | micro_recall    | 0.62     |
|  6 | micro_f1        | 0.62     |
+----+-----------------+----------+
------------------------------------------------------------


Evaluating:  29%|██▊       | 2/7 [00:00<00:00, 10.38it/s]

...

Execute [eval] method costing 699.10 ms



Evaluating:  14%|█▍        | 1/7 [00:00<00:00,  9.07it/s]

Execute [train_an_epoch] method costing 16842.01 ms
[Epoch 8] loss: 6.01769096404314



Epoch:  80%|████████  | 8/10 [02:33<00:38, 19.13s/it]

Execute [eval] method costing 699.31 ms
Early Stopping. Epoch: 8, best_valid_metric (macro_f1): 0.5440158952781674


In [9]:
cls.test(test_set)

Evaluating: 100%|██████████| 7/7 [00:00<00:00, 10.40it/s]

Execute [eval] method costing 677.06 ms


{'accuracy': 0.8415841584158416,
 'macro_precision': 0.5433262711864407,
 'macro_recall': 0.4935515873015873,
 'macro_f1': 0.4968821510297483,
 'micro_precision': 0.8415841584158416,
 'micro_recall': 0.8415841584158416,
 'micro_f1': 0.8415841584158416}

In [10]:
test_set.iloc[1:10]

,index,docs,labels,source,flag
903,290,\n Venezuelan President to Visit Vietnam from...,0,Biocaster,test
904,278,\n Bird flu outbreaks found in two more provi...,3,Biocaster,test
905,508,"\n KENYA: Eight dead, scores injured as build...",0,Biocaster,test
906,393,\n Indonesia cannot rule out human-to-human t...,2,Biocaster,test
907,149,\nDa Vinci Code falls under the censors knife ...,0,Biocaster,test
908,549,\n Wait Monument getting back to Springfield ...,0,Biocaster,test
909,968,\nDanger drugs stolen\nDANGEROUS drugs were st...,0,Biocaster,test
910,121,\n 09 Mar 2006\n Fatal Case of Meningitis De...,3,Biocaster,test
911,69,\n Legionnaires bacteria found in Calgary hos...,3,Biocaster,test


In [11]:
cls.predict(test_set.iloc[1:10])

Evaluating: 100%|██████████| 1/1 [00:00<00:00, 17.73it/s]

Execute [predict] method costing 76.85 ms


array([0, 3, 0, 0, 0, 0, 0, 0, 3])

### 